In [19]:
#best working version
#requires manual changes to the paths and param
import cv2 
import matplotlib.pyplot as plt
import os 
import glob 
import numpy as np
import math
from PIL import Image

#global var to keep track of base
def meanAbs(diffImg):
    sum =0 
    height = diffImg.shape[0]
    width = diffImg.shape[1]
    for row in range(0, height):
        for col in range(0, width):
            delta = int(diffImg[row,col].all())
            delta = delta * delta
            sum = sum + delta
    return math.sqrt(sum)/(width*height)
    
def imageDiff(image1, image2):
    dimensions = image1.shape
    height = image1.shape[0]
    width = image1.shape[1]
    diff = np.int32(image1) 
    
    for row in range(0, height):
        for col in range(0, width):
            if image1[row, col].all() > image2[row, col].all():
                diff[row, col] = image1[row, col]- image2[row, col]
            else:
                diff[row, col] = image2[row, col] -image1[row, col]

    return diff
    

def makeHough(img, scale):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray, 50,275)
    lines = cv2.HoughLines(edges, 1, np.pi / 180, 100, None, 0, 0)
    color = cv2.cvtColor(edges, cv2.COLOR_GRAY2BGR)
    if lines is not None:
        for i in range(0, len(lines)):
            rho = lines[i][0][0]
            theta = lines[i][0][1]
            a = math.cos(theta)
            b = math.sin(theta)
            x0 = a * rho
            y0 = b * rho
            pt1 = (int(x0 + 1000*(-b)), int(y0 + 1000*(a)))
            pt2 = (int(x0 - 1000*(-b)), int(y0 - 1000*(a)))
            cv2.line(img, pt1, pt2, (0,0,255), 1, cv2.LINE_AA)

    return img

def makeHCircles(img, scale):
    output = img.copy()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    blur = cv2.medianBlur(gray, 5)  
    detected_circles = cv2.HoughCircles(blur, cv2.HOUGH_GRADIENT, 1, 50, 20, 15, 1, 1) 
    #output = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
    # Draw circles that are detected. 
    if detected_circles is not None: 
           detected_circles = np.uint16(np.around(detected_circles)) 
    # Convert the circle parameters a, b and r to integers. 
           for pt in detected_circles[0, :]: 
                a, b, r = pt[0], pt[1], pt[2] 

            # Draw the circumference of the circle. 
                cv2.circle(output, (a, b), r, (0, 255, 0), 2) 
    return output
            
    
img_dir = "C:\\Users\\Steven\\Pictures\\CSCI412\\300px" #dir for img
data_path = os.path.join(img_dir,'*.png') #get all the .png files should be 55
files = glob.glob(data_path) 
data = [] 
circle = []
#store images into array
for i in range (0,30): 
    x = cv2.imread('C:\\Users\\Steven\\Pictures\\CSCI412\\300px\\'+str(i)+'.png')
    data.append(x)

#print images from array 
old = []
normalHough = []



for i in data:
       old.append(i.copy())

for x in data:
    x = makeHough(x)


for i in data:
    normalHough.append(i.copy())

    
for z in old:
    circle.append(makeHCircles(z))

#https://gist.github.com/mstankie/71e49f628beac320953e0460b8ee78c2
#https://www.geeksforgeeks.org/how-to-load-and-save-3d-numpy-array-to-file-using-savetxt-and-loadtxt-functions/
for i in range (30):  
    base = np.int16(old[i])
    circleH = np.int16(circle[i])
    line = np.int16(normalHough[i])
    diff = imageDiff(base, line)  #pass arrays
    res = np.hstack((old[i], normalHough[i]))
    cv2.imwrite('lines/100/Image' +str(i) + '.jpg',res)
    lineDiff = np.hstack(diff)
    cv2.imwrite('lines/100diff/Image' +str(i) + '.jpg',lineDiff)
    measure = meanAbs(diff);
    print("Image " + str(i) + " Mean Absolute lines = ", measure)
    res2 = np.hstack((old[i], circle[i] ))
    cv2.imwrite('circle/dist50/Image' +str(i) + '.jpg',res2)
    diff2 = imageDiff(base, circleH)
    circleDiff = np.hstack((circleH, diff2)) 
    cv2.imwrite('circle/diff50/Image' +str(i) + '.jpg',circleDiff)
    measureC = meanAbs(diff2);
    print("Image " + str(i) + " Mean Absolute circle = ", measureC)
    

       


Image 0 Mean Absolute circle =  0.001698051420478306
Image 1 Mean Absolute circle =  0.0016525658646177656
Image 2 Mean Absolute circle =  0.0019351284769291734
Image 3 Mean Absolute circle =  0.0018125609587213703
Image 4 Mean Absolute circle =  0.001622155630292263
Image 5 Mean Absolute circle =  0.0018549150845383999
Image 6 Mean Absolute circle =  0.0018643449350722314
Image 7 Mean Absolute circle =  0.0016376212809555324
Image 8 Mean Absolute circle =  0.0016352904618364805
Image 9 Mean Absolute circle =  0.0017396519447547226
Image 10 Mean Absolute circle =  0.001797843599999831
Image 11 Mean Absolute circle =  0.0017931434721348452
Image 12 Mean Absolute circle =  0.0017657230184198703
Image 13 Mean Absolute circle =  0.001705465071794527
Image 14 Mean Absolute circle =  0.0018072372929723423
Image 15 Mean Absolute circle =  0.0016467982403574654
Image 16 Mean Absolute circle =  0.0016413049778083069
Image 17 Mean Absolute circle =  0.0017822426571286216
Image 18 Mean Absolute c